# apply filter to images

In [1]:
import cv2
import numpy as np
import os

def detect_green_areas(input_dir, output_dir, min_area_fraction=0.001):
    # Ensure output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    counter = 1
    # Loop through every file in the input directory
    for filename in os.listdir(input_dir):

        if filename.endswith(('.png', '.jpg', '.jpeg','.tif','.JPG')):  # You can add more file extensions if needed
            image_path = os.path.join(input_dir, filename)
            
            # Read the image
            image = cv2.imread(image_path)

            # Convert image to HSV
            hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

            # Define the lower and upper bounds for green color
            lower_green = np.array([27, 30, 30])
            upper_green = np.array([90, 255, 255])

            # Create a binary mask where green regions are white
            mask = cv2.inRange(hsv, lower_green, upper_green)

            # Morphological operations to remove noise
            kernel = np.ones((3, 3), np.uint8)
            mask = cv2.erode(mask, kernel, iterations=1)
            mask = cv2.dilate(mask, kernel, iterations=1)
            mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
            mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
            

            # Calculate minimum area based on image dimensions
            min_area = min_area_fraction * image.shape[0] * image.shape[1]

            # Find connected components and their statistics
            num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(mask, connectivity=8)

            # Filter out components based on their area
            for i in range(1, num_labels):  # Exclude background label (0)
                if stats[i, cv2.CC_STAT_AREA] < min_area:
                    mask[labels == i] = 0  # Set pixels of small components to 0

            # Create a white background image
            white_background = np.ones_like(image) * 255

            # Merge the original image and the white background using the mask
            result = cv2.bitwise_and(image, image, mask=mask)
            result += cv2.bitwise_and(white_background, white_background, mask=cv2.bitwise_not(mask))

            # Save the result to the output directory
            output_path = os.path.join(output_dir, filename)
            cv2.imwrite(output_path, result)
            print(counter)
            counter += 1

# Paths to the directories
input_directory = r"D:\Projects\Weed Detection\mohab data\to be done\maize\Maize"
output_directory = r"D:\Projects\Weed Detection\mohab data\to be done\maize\processed"

# Call the function with min_area_fraction set to 0.001 (adjust as needed)
detect_green_areas(input_directory, output_directory, min_area_fraction=0.001)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [4]:
import cv2
import numpy as np
import os

def extract_and_save_largest_plant(image_path, output_path):
    # Read the image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Image at {image_path} could not be read.")
        return

    # Convert to grayscale and threshold to isolate plants
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY_INV)

    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        # Find the largest contour based on area
        largest_contour = max(contours, key=cv2.contourArea)

        # Create a mask for the largest contour
        mask = np.zeros_like(gray)  # Create a black image of the same dimensions as grayscale image
        cv2.drawContours(mask, [largest_contour], -1, (255), thickness=cv2.FILLED)

        # Apply the mask to the original image to isolate the largest plant
        masked_image = cv2.bitwise_and(image, image, mask=mask)

        # Make the non-plant areas white instead of black
        masked_white_bg = np.where(mask[:,:,None] == 0, 255, masked_image)

        # Crop the image based on the bounding rectangle of the largest contour
        x, y, w, h = cv2.boundingRect(largest_contour)
        cropped_image = masked_white_bg[y:y+h, x:x+w]

        # Save the cropped image
        cv2.imwrite(output_path, cropped_image)
        print(f"Saved the largest plant image to {output_path}")
    else:
        print(f"No contours found in {image_path}")

def process_folder(input_folder, output_folder):
    # Make sure output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # List all files in the input folder
    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff')):
            image_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, 'largest_' + filename)
            extract_and_save_largest_plant(image_path, output_path)

# Example usage
input_folder_path = r"D:\Projects\Weed Detection\mohab data\to be done\maize\processed"
output_folder_path = r"D:\Projects\Weed Detection\mohab data\to be done\maize\cut"
process_folder(input_folder_path, output_folder_path)


Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_20160628_081200.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_20160628_081203.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_20160628_081205.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_20160628_081210.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_20160628_081212.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_20160628_081216.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_20160628_081226.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_20160628_081228.jpg
Saved th

Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_20160628_081804.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_20160628_081807.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_20160628_081818.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_20160628_081823.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_20160628_081826.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_20160628_081837.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_20160628_081840.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_20160628_081845.jpg
Saved th

Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_20160628_082651.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_20160628_082653.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_20160628_082657.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_20160628_082659.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_20160628_082706.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_20160628_082708.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_20160628_082719.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_20160628_082720.jpg
Saved th

Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_2783.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_2784.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_2785.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_2786.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_2787.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_2788.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_2789.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_2790.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\larg

Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_2878.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_2879.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_2881.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_2882.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_2883.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_2884.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_2885.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_2886.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\larg

...

Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_3342.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_3343.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_3344.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_3345.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_3346.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_3347.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_3348.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_3349.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\larg

Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_3435.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_3436.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_3437.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_3438.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_3439.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_3440.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_3441.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_3442.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\larg

Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_3526.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_3527.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_3528.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_3529.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_3530.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_3532.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_3533.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_IMG_3534.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\larg

Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_STA40828.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_STA40829.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_STA40830.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_STA40831.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_STA40832.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_STA40833.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_STA40835.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_STA40836.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\larg

Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_STA40946.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_STA40947.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_STA40968.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_STA40969.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_STA40985.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_STA40986.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_STA40987.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\largest_STA40988.jpg
Saved the largest plant image to D:\Projects\Weed Detection\mohab data\to be done\maize\cut\larg